In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.metrics import mean_squared_error
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/gdsc-aiml-member-recruitments/train.csv')
test = pd.read_csv('/kaggle/input/gdsc-aiml-member-recruitments/test.csv')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

no null values present

In [ ]:
for i in df.columns:
    sns.scatterplot(df[i])
    plt.show()

### no features as showing any patterns

In [ ]:
for i in df.columns:
    sns.boxplot(df[i])
    plt.xlabel(i)
    plt.show()

### few varibales having some outliers this will not effect our models as we are using boosting techniques and tree regressor

In [ ]:
plt.figure(figsize=[12,12])
sns.heatmap(df.corr(),annot=True)
plt.show()

#### some varibales having high correlation

In [ ]:
x=df.drop(columns=['VACATION_RATE','QV2M','T2M'])
y=df['VACATION_RATE']

In [ ]:
for i in x.columns:
    sns.lineplot(x=df[i],y=y)
    plt.show()

### There are no liner releationship between any varibale so we cant use linear regression 

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=0)

In [ ]:
rf= RandomForestRegressor(random_state=10)

In [ ]:
rf.fit(xtrain,ytrain)

In [ ]:
pred=rf.predict(xtest)

In [ ]:
mean_squared_error(ytest,pred,squared=False)

In [ ]:
best=rf.feature_importances_

In [ ]:
new_fet=pd.DataFrame(data=best,index=x.columns)

In [ ]:
new_fet

### No we can eliminate Year as the importance is less

In [ ]:
x=df.drop(columns=['VACATION_RATE','QV2M','T2M','YEAR','ID'])
y=df['VACATION_RATE']

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
rf= RandomForestRegressor(random_state=10,min_samples_split=6,min_samples_leaf=4,max_depth=25,max_leaf_nodes=25,n_estimators=1550,max_features='log2')
rf.fit(xtrain,ytrain)
pred_y=rf.predict(xtest)
pred_x=rf.predict(xtrain)
a=mean_squared_error(ytest,pred_y,squared=False)
b=mean_squared_error(ytrain,pred_x,squared=False)
print(a,b)

### New algo Addaboost

In [ ]:
ad=AdaBoostRegressor(estimator=rf,learning_rate=0.5)
ad1=AdaBoostRegressor(RandomForestRegressor(random_state=10,min_samples_split=6,max_leaf_nodes=15,max_depth=16))

### ADA boosting with Randomforest

In [ ]:
ad.fit(xtrain,ytrain)
pred_y=ad.predict(xtest)
pred_x=ad.predict(xtrain)
a=mean_squared_error(ytest,pred_y,squared=False)
b=mean_squared_error(ytrain,pred_x,squared=False)
print(a,b)

In [ ]:
ad1.fit(xtrain,ytrain)
pred_y=ad1.predict(xtest)
pred_x=ad1.predict(xtrain)
a=mean_squared_error(ytest,pred_y,squared=False)
b=mean_squared_error(ytrain,pred_x,squared=False)
print(a,b)

In [ ]:
params = [{'learning_rate':[0.2],
          'loss':['exponential'],
          'n_estimators':[75,100]}]

In [ ]:
grid_ad1 = GridSearchCV(ad1,param_grid=params,cv=5,scoring='neg_root_mean_squared_error',verbose=2)
grid_ad1.fit(xtrain,ytrain)
grid_ad1.best_params_

In [ ]:
ad2 = AdaBoostRegressor(RandomForestRegressor(random_state=10,min_samples_split=6,max_leaf_nodes=15,max_depth=16),learning_rate=0.2,loss='exponential',n_estimators=75)

In [ ]:
ad2.fit(xtrain,ytrain)
pred_y=ad2.predict(xtest)
pred_x=ad2.predict(xtrain)
a=mean_squared_error(ytest,pred_y,squared=False)
b=mean_squared_error(ytrain,pred_x,squared=False)
print(a,b)

### Trying new model GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gbr=GradientBoostingRegressor(random_state = 10)

In [ ]:
param=[{
 'max_depth': [9,13],
 'max_features': ['log2','sqrt'],
 'max_leaf_nodes': [19,20],
 'min_samples_leaf': [4,5,7],
 'min_samples_split': [5,6,7],
 'n_estimators': [200,300]}]

In [ ]:
grid_gbr = GridSearchCV(gbr,param_grid=param,cv=5,scoring='neg_root_mean_squared_error',verbose=2)
grid_gbr.fit(xtrain,ytrain)
grid_gbr.best_params_

In [ ]:
gbr=GradientBoostingRegressor(random_state = 10,max_features='log2',max_leaf_nodes=15,min_samples_leaf=5,min_samples_split=8,n_estimators=300,max_depth=7)
gbr.fit(xtest,ytest)
pred_y=gbr.predict(xtest)
pred_x=gbr.predict(xtrain)
a=mean_squared_error(ytest,pred_y,squared=False)
b=mean_squared_error(ytrain,pred_x,squared=False)
print(a,b)

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb_p=XGBRegressor(n_estimators=150, max_depth=15, eta=0.1, subsample=0.7,max_leaves=8,gamma=3,random_state=10,learning_rate=0.1,booster='gbtree',n_jobs=0)
xgb_p.fit(xtrain,ytrain)
pred_y=xgb_p.predict(xtest)
pred_x=xgb_p.predict(xtrain)
a=mean_squared_error(ytest,pred_y,squared=False)
b=mean_squared_error(ytrain,pred_x,squared=False)
print(a,b)

In [ ]:
test_df = test.drop(columns=['YEAR','ID'])
predictions = xgb_p.predict(test_df)
# # Create a submission file
sub_file = pd.DataFrame({'ID': test.ID, 'VACATION_RATE': predictions})
sub_file.head()

In [ ]:
sub_file.to_csv('SubmissionTemp40.csv', index = False) # Download and submit, or submit via Notebook output